## 1:  `britts_in_harvard`

Write a program to fetch the names and birth places of all British persons listed in the API. Store your program in a publicly-accessible Jupyter notebook via Google Colab.





In [0]:
import requests

In [0]:
'''
need to generate an API by submitting 
a request at: https://docs.google.com/forms/d/e/1FAIpQLSfkmEBqH76HLMMiCC-GPPnhcvHC9aJS86E32dOd0Z8MpY2rvQ/viewform 
'''

api = 'fc2c1ee0-871e-11ea-901a-298e9b8e7864'


In [0]:
''' 
  purp: get names and birthplaces of ALL Britts from Harvard API. 
  
  Endpoint used: Person

  Query parameters available at: https://github.com/harvardartmuseums/api-docs/blob/master/sections/person.md
'''

# base URL then becomes
url = 'https://api.harvardartmuseums.org/person'

# better to have culture: British in the query itself
url = 'https://api.harvardartmuseums.org/person?q=culture:British'


In [0]:
# query: culture not working it seems, page works
query = {"apikey": api, "size": 100, "page": 1}  

# modify the query; add fields of interest; not working 
# query = {"apikey": api, "size": 100}
#         'fields':['culture', 'birthplace', 'displayname']}  




In [117]:
response = requests.get(url, query)
# is it a  good response?
response.ok

True

In [118]:
# what's in the data: instead of just Britts, it seems to have all persons. Can't understand why there are Nones in here 
data = response.json()
data.keys()

dict_keys(['info', 'records'])

In [143]:
# each record is saves as an entry in record. Can use indexing [1] to access the record 
# and the field of choice thereafter. That means, we could also use 'fields' as a query
# to get the desired fields: culture (to later extract British), displayname , and birthplace  
data['records'][1]

{'alphasort': 'Read, David Charles',
 'birthplace': None,
 'culture': 'British',
 'datebegin': 1790,
 'dateend': 1851,
 'deathplace': None,
 'displaydate': '1790 - 1851',
 'displayname': 'David Charles Read',
 'gender': 'unknown',
 'id': 18455,
 'lastupdate': '2020-04-25T05:10:57-0400',
 'objectcount': 1,
 'personid': 18455,
 'url': 'https://www.harvardartmuseums.org/collections/person/18455'}

In [122]:
data['info']

{'next': 'https://api.harvardartmuseums.org/person?q=culture%3ABritish&apikey=fc2c1ee0-871e-11ea-901a-298e9b8e7864&size=100&page=2',
 'page': 1,
 'pages': 19,
 'totalrecords': 1869,
 'totalrecordsperquery': 100}

In [0]:
# can print a max of 100 records using query size
if len(data["records"]) == 100:
  print(f'Can only print {len(data["records"])} records in one go.')

In [138]:
def process_data(raw_incidents):
  #iterate the results and only grab the properties that we are interetsed in
  return [{
      'name':raw_incidents[incident]['displayname'],
      'birthplace': raw_incidents[incident]['birthplace'],
      'nationality': raw_incidents[incident]['culture']
  } for incident in range(len(raw_incidents))]
len(process_data(data['records']))

99

In [141]:
'''
  raw_incidents: records of persons with culture=British or Britts 
  Lists of Dictionaries

'''
def process_data(raw_incidents):
  #iterate the results and only grab the properties that we are interetsed in
  return [{
      'name': raw_incidents[incident]['displayname'],
      'birthplace': raw_incidents[incident]['birthplace'],
      'nationality': raw_incidents[incident]['culture']
  } for incident in range(len(raw_incidents))]
  

# declare a list to store all results
incidents = []

# declare variables to track pages amount
page = 1

# make an initial call
url = 'https://api.harvardartmuseums.org/person?q=culture:British'
query = {"apikey": api, "size": 100, "page": page}  
response = requests.get(url, query)
print('Querying {}'.format(response.url))

# make sure we got a valid response
if(response.ok):
  # get the full data from the response
  data = response.json()
  
  # get the meta data
  meta_data = data['info']

  total = meta_data['totalrecords']
  print('There is a total of {} results to fetch'.format(total))

  total_pages = meta_data['pages']
  print('There is a total of {} pages to process'.format(total_pages))
  
  # process the results we have so far
  incidents = process_data(data['records'])
  print('{} results processed so far'.format(len(incidents)))
  
  # increment skip
  page = page + 1
  
  while page <= total_pages:
    query = {"apikey": api, "size": 100, "page": page} 
    response = requests.get(url, query)
    print('Querying {}'.format(response.url))
    if(response.ok):
      #  now incidents will be the old values plus the new ones      
      incidents = incidents + process_data(response.json()['records'])
      print('{} results processed so far'.format(len(incidents)))
      # increment skip
      page = page + 1
      
print('{} results returned'.format(len(incidents)))      



Querying https://api.harvardartmuseums.org/person?q=culture:British&apikey=fc2c1ee0-871e-11ea-901a-298e9b8e7864&size=100&page=1
There is a total of 1869 results to fetch
There is a total of 19 pages to process
100 results processed so far
Querying https://api.harvardartmuseums.org/person?q=culture:British&apikey=fc2c1ee0-871e-11ea-901a-298e9b8e7864&size=100&page=2
200 results processed so far
Querying https://api.harvardartmuseums.org/person?q=culture:British&apikey=fc2c1ee0-871e-11ea-901a-298e9b8e7864&size=100&page=3
300 results processed so far
Querying https://api.harvardartmuseums.org/person?q=culture:British&apikey=fc2c1ee0-871e-11ea-901a-298e9b8e7864&size=100&page=4
400 results processed so far
Querying https://api.harvardartmuseums.org/person?q=culture:British&apikey=fc2c1ee0-871e-11ea-901a-298e9b8e7864&size=100&page=5
500 results processed so far
Querying https://api.harvardartmuseums.org/person?q=culture:British&apikey=fc2c1ee0-871e-11ea-901a-298e9b8e7864&size=100&page=6
600 r